<a href="https://colab.research.google.com/github/JP109/Tensorflow-Basics/blob/main/One_Device_Strategy_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is typically used to deliberately test your code on a single device. This can be used before switching to a different strategy that distributes across multiple devices.

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

## **Define the Distribution Strategy**

You can list available devices in your machine and specify a device type. This allows you to verify the device name to pass in tf.distribute.OneDeviceStrategy().

In [ ]:
# choose a device type such as CPU or GPU
devices = tf.config.list_physical_devices('GPU')
print(devices[0])

# You'll see that the name will look something like "/physical_device:GPU:0"
# Just take the GPU:0 part and use that as the name
gpu_name = "GPU:0"

# define the strategy and pass in the device name
one_strategy = tf.distribute.OneDeviceStrategy(device=gpu_name)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## **Global variables**

In [ ]:
pixels = 224
MODULE_HANDLE = 'https://tfhub.dev/tensorflow/resnet_50/feature_vector/1'
IMAGE_SIZE = (pixels, pixels)
BATCH_SIZE = 32

print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

Using https://tfhub.dev/tensorflow/resnet_50/feature_vector/1 with input size (224, 224)


## **Download and Prepare the Dataset**

In [ ]:
splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']

(train_examples, validation_examples, test_examples), info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True, split=splits)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteQ05OZM/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [ ]:
# resize the image and normalize pixel values
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

In [ ]:
# prepare batches
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [ ]:
# check if the batches have the correct size and the images have the correct shape
for image_batch, label_batch in train_batches.take(1):
    pass

print(image_batch.shape)

(32, 224, 224, 3)


## **Define and Configure the Model**

In [ ]:
# tells if we want to freeze the layer weights of our feature extractor during training
do_fine_tuning = False

In [ ]:
def build_and_compile_model():
    print("Building model with", MODULE_HANDLE)

    # configures the feature extractor fetched from TF Hub
    feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   trainable=do_fine_tuning)

    # define the model
    model = tf.keras.Sequential([
      feature_extractor,
      # append a dense with softmax for the number of classes
      tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    # display summary
    model.summary()

    # configure the optimizer, loss and metrics
    optimizer = tf.keras.optimizers.SGD(lr=0.002, momentum=0.9) if do_fine_tuning else 'adam'
    model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

In [ ]:
# build and compile under the strategy scope
with one_strategy.scope():
    model = build_and_compile_model()

Building model with https://tfhub.dev/tensorflow/resnet_50/feature_vector/1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              23561152  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,565,250
Trainable params: 4,098
Non-trainable params: 23,561,152
_________________________________________________________________


In [ ]:
EPOCHS = 5
hist = model.fit(train_batches,
                 epochs=EPOCHS,
                 validation_data=validation_batches)

Epoch 1/5
582/582 [==============================] - 105s 114ms/step - loss: 0.0397 - accuracy: 0.9848 - val_loss: 0.0369 - val_accuracy: 0.9871
Epoch 2/5
582/582 [==============================] - 69s 112ms/step - loss: 0.0202 - accuracy: 0.9933 - val_loss: 0.0270 - val_accuracy: 0.9905
Epoch 3/5
582/582 [==============================] - 70s 114ms/step - loss: 0.0144 - accuracy: 0.9953 - val_loss: 0.0283 - val_accuracy: 0.9910
Epoch 4/5
582/582 [==============================] - 70s 113ms/step - loss: 0.0115 - accuracy: 0.9963 - val_loss: 0.0270 - val_accuracy: 0.9927
Epoch 5/5
582/582 [==============================] - 70s 113ms/step - loss: 0.0103 - accuracy: 0.9969 - val_loss: 0.0293 - val_accuracy: 0.9931
